In [1]:
!git clone https://github.com/AleksL04/ML-Project.git
%cd ML-Project

Cloning into 'ML-Project'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 127 (delta 59), reused 111 (delta 43), pack-reused 0 (from 0)
Receiving objects: 100% (127/127), 50.04 KiB | 5.56 MiB/s, done.
Resolving deltas: 100% (59/59), done.
/content/ML-Project


In [2]:
!git pull

Already up to date.


In [3]:
%%capture
!pip install --upgrade gensim
!gdown --folder 1rl_TJMHtcP-S0fTHNUpr9oi8Ndz2Y5z4 -O project_data
!gdown --folder 1RmC_We2lfsJontwrxlc4dzqEY_mNy-UW -O word2vec_model

!pip install --upgrade scikeras
!pip install "scikit-learn<1.6"


In [4]:
ls -l

total 28
-rw-r--r-- 1 root root 10990 Dec 12 01:03 main_console.ipynb
drwxr-xr-x 2 root root  4096 Dec 12 01:03 model_code/
-rw-r--r-- 1 root root  1550 Dec 12 01:03 process_df.py
drwxr-xr-x 2 root root  4096 Dec 12 01:03 project_data/
drwxr-xr-x 2 root root  4096 Dec 12 01:03 word2vec_model/


In [5]:
import pandas as pd
import numpy as np

In [6]:
train_df = pd.read_csv('project_data/train.csv')
valid_df = pd.read_csv('project_data/valid.csv')
test_df = pd.read_csv('project_data/test.csv')

In [7]:
from process_df import process_df

X_train = process_df(train_df)
y_train = train_df['label'].values

X_valid = process_df(valid_df)
y_valid = valid_df['label'].values

X_test = process_df(test_df)
y_test = test_df['label'].values

In [8]:
from model_code.LSTM_code import trainLSTM, train_CV_LSTM
from model_code.CNN_code import trainCNN, train_CV_CNN
from model_code.Blender_code import trainBlender
from model_code.Bayes_code import trainBayes, predictBayes

In [9]:
# train_CV_LSTM(X_train, y_train, X_valid, y_valid)
#train_CV_CNN(X_train, y_train, X_valid, y_valid)

In [10]:
LSTM_model = trainLSTM(X_train, y_train, X_valid, y_valid)

test_loss, test_acc = LSTM_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8980 - loss: 0.2517
Test Accuracy: 0.8903


In [11]:
CNN_model = trainCNN(X_train, y_train, X_valid, y_valid)

test_loss, test_acc = CNN_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8553 - loss: 0.3360
Test Accuracy: 0.8634


In [12]:
Bayes_model, vectorizer = trainBayes(train_df, valid_df)

Validation Accuracy: 0.7541899441340782
Validation Loss: 0.5015656617689024


In [13]:
val_pred_LSTM = LSTM_model.predict(X_test).flatten()
val_pred_CNN = CNN_model.predict(X_test).flatten()
_,_,val_pred_Bayes = predictBayes(Bayes_model, valid_df, vectorizer)

X_meta_train = np.column_stack([
    val_pred_LSTM,
    val_pred_CNN,
    val_pred_Bayes
])

y_meta_train = y_valid

31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


NameError: name 'X' is not defined

In [ ]:
# val_pred_LSTM = LSTM_model.predict(X_valid)
# val_pred_CNN = CNN_model.predict(X_valid)
# val_pred_Bayes = predictBayes(Bayes_model, valid_df, vectorizer)
# print(val_pred_Bayes)
# print(val_pred_LSTM)
# X_meta_train = np.column_stack((val_pred_LSTM, val_pred_CNN))

# y_meta_train = y_valid

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


AttributeError: 'tuple' object has no attribute 'predict'

In [ ]:
Blender_model = trainBlender(X_meta_train, y_meta_train)

In [ ]:
test_pred_LSTM = LSTM_model.predict(X_test)
test_pred_CNN = CNN_model.predict(X_test)

X_meta_test = np.column_stack((test_pred_LSTM, test_pred_CNN))

test_loss, test_acc = Blender_model.evaluate(X_meta_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")